Notebook for viewing detailed activity data.

Data obtained by requesting a comprehensive data export from Garmin (https://www.garmin.com/en-US/account/datamanagement/exportdata).

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import os, sys
import pandas as pd
import json

datadir = 'data'

def digital_time_to_minutes(timestr):
    to_minutes = [60,1,1/60]
    hrs,mins,secs = [conv*float(x) for x,conv in zip(timestr.split(':'), to_minutes)]
    minutes = hrs + mins + secs
    return minutes

## explore Garmin data file structure - run first

In [3]:
os.listdir('./data/')

['Activities_20250110.csv', 'garmin_data_20250114', 'garmin_data_20250114.zip']

In [4]:
connect_dir = './data/garmin_data_20250114/DI_CONNECT'
os.listdir(connect_dir)

['DI-Connect-Aggregator',
 'DI-Connect-Fitness',
 'DI-Connect-Metrics',
 'DI-Connect-Uploaded-Files',
 'DI-Connect-User',
 'DI-Connect-Wellness',
 'DI-GOLF']

### Wellness

In [5]:
subdir = os.path.join(connect_dir, 'DI-Connect-Wellness')
files = os.listdir(subdir)
sizes = [os.path.getsize(os.path.join(subdir,f)) for f in files]
for f,s in zip(files,sizes):
    print(f,s, "B")

116924331_bioMetrics_latest.json 4 B
116924331_heartRateZones.json 213 B
116924331_userBioMetricProfileData.json 62 B
116924331_userBioMetrics.json 74648 B
2023-08-30_2023-12-08_116924331_sleepData.json 34908 B
2023-12-08_2024-03-17_116924331_sleepData.json 57275 B
2024-03-17_2024-06-25_116924331_sleepData.json 57596 B
2024-06-25_2024-10-03_116924331_sleepData.json 59062 B
2024-10-03_2025-01-11_116924331_sleepData.json 60521 B


In [6]:
fname = '2024-10-03_2025-01-11_116924331_sleepData.json'
with open(os.path.join(subdir, fname)) as f:
    flist = json.load(f)

for ls in flist:
    print(list(ls.keys()))

['sleepStartTimestampGMT', 'sleepEndTimestampGMT', 'calendarDate', 'sleepWindowConfirmationType', 'deepSleepSeconds', 'lightSleepSeconds', 'remSleepSeconds', 'awakeSleepSeconds', 'unmeasurableSeconds', 'spo2SleepSummary', 'averageRespiration', 'lowestRespiration', 'highestRespiration', 'retro']
['sleepStartTimestampGMT', 'sleepEndTimestampGMT', 'calendarDate', 'sleepWindowConfirmationType', 'deepSleepSeconds', 'lightSleepSeconds', 'remSleepSeconds', 'awakeSleepSeconds', 'unmeasurableSeconds', 'spo2SleepSummary', 'averageRespiration', 'lowestRespiration', 'highestRespiration', 'retro']
['sleepStartTimestampGMT', 'sleepEndTimestampGMT', 'calendarDate', 'sleepWindowConfirmationType', 'deepSleepSeconds', 'lightSleepSeconds', 'remSleepSeconds', 'awakeSleepSeconds', 'unmeasurableSeconds', 'spo2SleepSummary', 'averageRespiration', 'lowestRespiration', 'highestRespiration', 'retro']
['sleepStartTimestampGMT', 'sleepEndTimestampGMT', 'calendarDate', 'sleepWindowConfirmationType', 'deepSleepSeco

## Sleep Data

In [14]:
subdir = os.path.join(connect_dir, 'DI-Connect-Wellness')
files = os.listdir(subdir)
sizes = [os.path.getsize(os.path.join(subdir,f)) for f in files]
sleep_files = []
for f,s in zip(files,sizes):
    print(f,s, "B")
    if 'sleepData' in f:
        sleep_files.append(f)
print(sleep_files)

116924331_bioMetrics_latest.json 4 B
116924331_heartRateZones.json 213 B
116924331_userBioMetricProfileData.json 62 B
116924331_userBioMetrics.json 74648 B
2023-08-30_2023-12-08_116924331_sleepData.json 34908 B
2023-12-08_2024-03-17_116924331_sleepData.json 57275 B
2024-03-17_2024-06-25_116924331_sleepData.json 57596 B
2024-06-25_2024-10-03_116924331_sleepData.json 59062 B
2024-10-03_2025-01-11_116924331_sleepData.json 60521 B
['2023-08-30_2023-12-08_116924331_sleepData.json', '2023-12-08_2024-03-17_116924331_sleepData.json', '2024-03-17_2024-06-25_116924331_sleepData.json', '2024-06-25_2024-10-03_116924331_sleepData.json', '2024-10-03_2025-01-11_116924331_sleepData.json']


In [17]:
type(files)

str

In [18]:
fname = '2024-10-03_2025-01-11_116924331_sleepData.json'

content_list = []
for fname in sleep_files:
    with open(os.path.join(subdir, fname)) as f:
        content_list += json.load(f)

In [20]:
content_list[0]['sleepStartTimestampGMT'], content_list[0]['sleepEndTimestampGMT']

('2023-10-11T04:00:00.0', '2023-10-11T12:00:00.0')

In [21]:
sleeptimestamp = content_list[0]['sleepStartTimestampGMT']
print(sleeptimestamp.split('T'))

sleeptimestamp = content_list[0]['sleepEndTimestampGMT']
print(sleeptimestamp.split('T'))

['2023-10-11', '04:00:00.0']
['2023-10-11', '12:00:00.0']


In [ ]:
# create a dictionary of sleep data by date
